In [1]:
!pip install outlines
!pip install context_cite

You should consider upgrading via the 'E:\kaggle_comps\predict_future_sales\code\venv\new-venv\Scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'E:\kaggle_comps\predict_future_sales\code\venv\new-venv\Scripts\python.exe -m pip install --upgrade pip' command.


## Imports

In [2]:
%load_ext autoreload
%autoreload 2

# Add the path to the parent directory to sys
import sys, os

# If current directory is called 'notebooks', chdir to the parent
if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir('../')
    
sys.path.append('attribution')

from torch.utils.data import DataLoader

import pandas as pd
from constants import ModelNames, DatasetNames, LANGUAGE_MAPPING
from model_utils import Model 
from dataset_utils import GSMDataset, PaddingCollator, is_correct_gsm, extract_answer_gsm
from context_cite import ContextCiter

e:\kaggle_comps\predict_future_sales\code\venv\new-venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


## Load Necessary Codes

In [3]:
def load_model_dataset():
    model = Model(ModelNames.QwenInstruct)
    
    return model, DatasetNames.MGSM

## Analysis: Processing Responses

In [4]:
class ResponseProcessing():
    def __init__(self, model, dataset):
        self.df_column_names = ["question", "actual_answer", "model_gen_steps", "model_gen_answer" ]
        self.model = model
        self.dataset = dataset
    
    def convert_dashes_incremental_steps_list(self, step):
        furnished_steps = []

        i = 1
        for _, step in enumerate(steps[1:]):
            if step:  # Skip empty parts (if any)
                furnished_steps.append(str(i) + ". " + step )  # Replace with number (1, 2, 3...)
                i += 1
        
        return furnished_steps

    def convert_dashes_incremental_steps(self, step):

        '''
        Returns str
        '''

        furnished_steps = self.convert_dashes_incremental_steps_list(step)

        final_str = "Step-by-Step Answer:\n"

        final_str += "".join(furnished_steps)

        return final_str


    def process_model_responses_for_analysis(self):

        mgsm_test = GSMDataset(self.dataset, self.model.tokenizer, instructions='', split='test', config='en')

        mgsm_generation_df = pd.read_csv('results\mgsm_en_Qwen2.5-1.5B-Instruct_results.csv')
        mgsm_generations = mgsm_generation_df['response'].tolist()

        all_steps = []
        all_gen_final_ans = []

        for response in mgsm_generations:
            
            steps = response.split("\n-")
            final_step = steps[-1].split(".\r\n")[0]
            steps.pop()
            steps.append( final_step )

            steps_str = self.convert_dashes_incremental_steps(steps)
            all_steps.append( steps_str )

            gen_final_ans = extract_answer_gsm(response)
            all_gen_final_ans.append( gen_final_ans )

        question_list = mgsm_test.dataset['question']
        actual_answer = mgsm_test.dataset['answer_number']

        percentile_list = pd.DataFrame(data=zip(question_list,actual_answer,all_steps, all_gen_final_ans), columns=self.df_column_names)

        percentile_list.to_csv("results/analysis_mgsm_en_Qwen2.5-1.5B-Instruct_results.csv", index = False)



## Main

In [ ]:
if '__main__':
    model, dataset = load_model_dataset()
    
    responseProcessing = ResponseProcessing(model, dataset)
    responseProcessing.process_model_responses_for_analysis()

    

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


KeyboardInterrupt: 

## Inferencing

### Steps:
 1. read from "analysis_{model_name}"
 2. pass in model_generated_steps and query
 3. Check if there answer matches with our answer
 4. If yes, then use cc.getattribution() to attribution [contextCite](https://github.com/MadryLab/context-cite)
 5. Save the np.array to the respective row of the "analysis_{model_name}" set

In [ ]:
model_name = "Qwen/Qwen2.5-1.5B-Instruct"

## step
context = """
" Step-by-Step Answer:
1.  First, calculate the number of eggs laid by the ducks per day: 16 eggs/day.
2.  Janet eats 3 eggs for breakfast every morning, so subtract those: 16 - 3 = 13 eggs remaining.
3.  She also bakes muffins for her friends using 4 eggs, so again subtract those: 13 - 4 = 9 eggs remaining.
4.  Janet sells these remaining eggs at the farmers' market for $2 per egg, so multiply the number of eggs by this price: 9 * $2 = $18 per day"
"""

## model_generated_answer
query = "Janet's ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?"

In [ ]:
cc = ContextCiter.from_pretrained(model_name, context, query, device="cuda")

AssertionError: Torch not compiled with CUDA enabled